## Imports

In [12]:
import pandas as pd
import ast
import math
import math
import numpy as np
from math import pi
from bounding import MinimumBoundingBox
from sympy import symbols, Eq, solve

In [2]:
data = pd.read_csv('Crimes_-_2018.csv')

In [3]:
filtered_data = data[['Primary Type', 'Location']].dropna()

In [4]:
filtered_data['row_num'] = [i for i in range(1, len(filtered_data) + 1)]

In [5]:
filtered_data

,Primary Type,Location,row_num
152,OTHER OFFENSE,"(41.896063478, -87.667902742)",1
154,ASSAULT,"(41.769399886, -87.690705813)",2
155,BATTERY,"(41.838706452, -87.72595418)",3
156,OFFENSE INVOLVING CHILDREN,"(41.896551396, -87.773787663)",4
425,BATTERY,"(42.004384227, -87.674295522)",5
...,...,...,...
268672,ASSAULT,"(41.897598907, -87.769139392)",263467
268673,ROBBERY,"(41.732730806, -87.585114125)",263468
268674,ROBBERY,"(41.734234372, -87.589291682)",263469
268681,OFFENSE INVOLVING CHILDREN,"(41.732935859, -87.608977015)",263470


In [6]:
robery_data = filtered_data[filtered_data['Primary Type'] == 'ROBBERY']

In [44]:
def get_elipse_radii(center, p1, p2):
    x, y = symbols('x y')
    eq1 = Eq(((p1[0] - center[0]) ** 2)*x + ((p1[1] - center[1]) ** 2)*y - 1, 0)
    eq2 = Eq(((p2[0] - center[0]) ** 2)*x + ((p2[1] - center[1]) ** 2)*y - 1, 0)
    ans = solve((eq1, eq2), (x, y))
    print(ans)
    r1, r2 = ans[x], ans[y]
    print(float(r1), float(r2))
    r1, r2 = (1 / float(r1)) ** (0.5), (1 / float(r2)) ** (0.5)
    
    return r1, r2

In [45]:
def step_1(data_array):
    possible_elipses = set()
    for i in data_array:
        for j in data_array:
            for k in data_array:
                if i[2] != j[2] and j[2] != k[2] and i[2] != k[2]:
                    loc = ast.literal_eval(i[1])
                    radii = get_elipse_radii(loc, ast.literal_eval(j[1]), ast.literal_eval(k[1]))
                    possible_elipses.add((loc[0],loc[1],radii[0], radii[1]))
            
    return possible_elipses

In [ ]:
def helper_function(h, k, x, y, a, b):
 
    # checking the equation of
    # ellipse with the given point
    p = ((math.pow((x - h), 2) / math.pow(a, 2)) +
         (math.pow((y - k), 2) / math.pow(b, 2)))
 
    if p <= 1:
        return True
    
def step_2(step_1_elipses, data_array,  monte_flag = False):
    elipse_likelihood = {}
    area_s = MinimumBoundingBox([ast.literal_eval(i[1]) for i in data_array]).area
    for i in step_1_elipses:
        B = 100*(pi*i[2]*i[3])/area_s
        curr_c = 0
        curr_i = 0
        for j in data_array:
            loc = ast.literal_eval(j[1])
            if helper_function(i[0], i[1], loc[0], loc[1], i[2], i[3]):
                curr_c += 1
        if curr_c > B:
            curr_i = 1
        if curr_i == 0:
            likelihood = 0
        else:
            likelihood = math.log(((curr_c/B)**curr_c)*(((100-curr_c)/(100-B))**(100-curr_c))*curr_i)
        elipse_likelihood[i] = likelihood
    final_elipses = {}
    max_like = float('-inf')
    max_like_elipse = None
    for i,j in elipse_likelihood.items():
        for a,b in elipse_likelihood.items():
            x = helper_function(i,j,b,a)
            if x != None and x[1] > MIN_LOG_LIKELIHOOD:
                final_elipses[x[0]] = x[1]
                if x[1] > max_like:
                    max_like_elipse = x[0]
    if monte_flag:
        return max_like_elipse, max_like
    fin = {'Elipse': [], 'Likelihood': []}
    for i, j in final_elipses.items():
        fin['Elipse'].append(i)
        fin['Likelihood'].append(j)
    pd.DataFrame(fin).to_csv('elipse_hood.csv')
    return final_elipses